# California Policing Scorecard

### Overview

The [California Police Scorecard](https://policescorecard.org) utilizes data on a range of policing-related issues to evaluate how each police department interacts with, and the extent to which officers are held accountable to, the communities they serve. The indicators included in this scorecard were selected based on a review of the [research literature](http://joincampaignzero.org/research), input from activists and experts in the field, and a review of existing publicly available datasets on policing in California. The scorecard is designed to help communities, researchers, police leaders and policy-makers take informed action to reduce police use of force and improve accountability and public safety in their jurisdictions. 

### About the Data
Data on police deadly force, civilian complaints and arrests that were obtained from official databases including the California Department of Justice’s [OpenJustice](https://openjustice.doj.ca.gov/data) database, UCR and the California Monthly Arrests and Citation Register. Data on police use of force, use of force complaints, and police policy manuals were obtained directly from police agencies via public records requests.


## Imports

In [1]:
import pandas
import numpy

## Display Settings

In [2]:
# Set display options to be able to view wide columns and many rows
pandas.set_option('display.max_columns', 500)
pandas.set_option('display.max_rows', 500)

## Data

In [3]:
raw = pandas.read_excel('data/raw/disparity analysis 2016-2018.xlsx')

In [4]:
raw.head()

,Agency Name,Asian / Pacific Islander Victims of Deadly Force,White Victims of Deadly Force,Black Victims of Deadly Force,Hispanic Victims of Deadly Force,Other Victims of Deadly Force,Percent Asian / Pacific Islander Deadly Force Victims,Percent White Deadly Force Victims,Percent Black Deadly Force Victims,Percent Hispanic Deadly Force Victims,Total Arrests 2016-2018,Asian/Pacific Islander Arrests,Black Arrests,Hispanic Arrests,White Arrests,Other Arrests,Percent Asian/Pacific Islander Arrests,Percent White Arrests,Percent Black Arrests,Percent Hispanic Arrests,Black Drug Possession Arrests,Hispanic Drug Possession Arrests,White Drug Possession Arrests,Black Drug Possession Arrest Disparity,Hispanic Drug Possession Arrest Disparity,Black Deadly Force per Arrest Rate,Black Deadly Force Disparity per Arrest,Black Deadly Force Disparity per Population,Hispanic Deadly Force per Arrest Rate,Hispanic Deadly Force Disparity per Arrest,Hispanic Deadly Force Disparity per Population,Total Population,Hispanic Population,White Population,Black Population,Asian / Pacific Islander Population,Other Population
0,Alameda Police Department,0,1,0,0,0,0.0000,1.000000,0.000000,0.000000,3581,263.418459,989.061763,755.464261,1322.062457,250.993060,0.073560,0.369188,0.276197,0.210965,17,25.0,49,1.897261,1.684697,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,80060,10066,33238,6078,25280,4992
1,Alhambra Police Department,0,0,0,2,0,0.0000,0.000000,0.000000,1.000000,3911,303.724254,279.514350,2806.148002,345.541362,176.072032,0.077659,0.088351,0.071469,0.717501,12,214.0,34,2.204300,1.768305,0.000000,NaN,NaN,7.127208,NaN,NaN,84694,32234,9056,1450,42248,1487
2,Anaheim Police Department,1,4,1,9,1,0.0625,0.250000,0.062500,0.562500,28200,787.346311,2075.512295,15479.661885,7464.139344,2393.340164,0.027920,0.264686,0.073600,0.548924,128,1253.0,652,2.703173,0.932133,4.818088,0.899072,3.442322,5.814080,1.084928,1.091332,352516,189491,91910,6675,60699,6116
3,Antioch Police Department,0,3,5,0,0,0.0000,0.375000,0.625000,0.000000,10115,149.306653,4260.496881,2044.029106,3398.303534,262.863825,0.014761,0.335967,0.421206,0.202079,153,65.0,169,1.293171,0.305692,11.735720,1.329385,2.380674,0.000000,0.000000,0.000000,111574,40000,31792,22257,11679,6902
4,Bakersfield Police Department,0,21,10,34,2,0.0000,0.313433,0.149254,0.507463,51337,474.048525,10191.098963,19530.421492,20459.632146,681.798874,0.009234,0.398536,0.198514,0.380436,754,1560.0,2656,1.166227,0.392177,9.812484,0.955999,1.956233,17.408738,1.696078,1.081049,390380,186871,124775,30373,28829,8893


# Log-normal distributions

Most of the time, whenever we want to analyze counts of something at the city level, the data is going to follow the Log-normal distribution i.e. population is concentrated in a few cities so drug possession arrests are also going to be concentrated in a few cities.

In [5]:
hist(raw['Black Drug Possession Arrests'].fillna(0), bins=25)

# Means and the Normal distribution

If we look at the mean of the distribution above, we get a high average for arrests, when it's clear that the majority of the data are closer to 0, but larger cities like Los Angeles or Oakland skew the mean upward.

In [6]:
raw['Black Drug Possession Arrests'].fillna(0).mean()

127.42

# Variance-stabilizing transform

The mean/standard deviation functions often used in statistics make the assumption that the data follows a Normal/Gaussian distribution, also known as a bell curve. When the histogram of our data looks very different from a bell curve, we need to apply a variance-stabilizing transform. In the case of the log-Normal distribution above, taking the log of the data gives us a normal distribution and puts smaller cities and larger cities on the same scale. Since the log of 0 is undefined, we add one to any zero counts, and since log(1) is zero, this has the effect of keeping the zero values zero after the transform.

In [7]:
hist(numpy.log(raw['Black Drug Possession Arrests'].fillna(0) + 1))

We can operate on the log-transformed scale for statistics like the mean and standard deviation, and take the exponent when we need to transform back into the original scale (for example, the formula below computes the log average).

In [8]:
numpy.exp(numpy.log(raw['Black Drug Possession Arrests'].fillna(0) + 1).mean())

64.20200975752935

# Population Statistics

Here we use the population statistics from the OpenJustice database (2016) multiplied by the proportions by race, rather than using the Census/ACS, to maintain consistency with the other scores that rely on these figures.

In [9]:
total_pop = raw['Total Population']
black_pop = raw['Black Population']
black_pct_pop = black_pop / total_pop
latinx_pop = raw['Hispanic Population']
latinx_pct_pop = latinx_pop / total_pop
white_pop = raw['White Population']
white_pct_pop = white_pop / total_pop

# Drug Arrest Disparities

Since [rates of drug use are similar across racial groups](https://store.samhsa.gov/system/files/nsduhresults2013.pdf), disparities in arrests for drug possession suggest racial bias in policing.

The disparities for Black and Latinx drug arrests are calculated using the following formulae for each municipal police department in California:

$$Black\ drug\ arrest\ disparity = \dfrac{(Black\ drug\ arrests\ /\ Black\ population)}{(White\ drug\ arrests\ /\ White\ population)}$$

$$Latinx\ drug\ arrest\ disparity = \dfrac{(Latinx\ drug\ arrests\ /\ Latinx\ population)}{(White\ drug\ arrests\ /\ White\ population)}$$

To reasonably compare disparities across different racial and ethnic groups, we want to ultimately transform the scores into a Normal/Gaussian distribution and use standard scores (also known as Z-scores). We use the logit or log-odds, which is a common transform for stabilizing the variance in data consisting of percentages. Starting from the initial per-group percentages for each department:

$$\%\ Black\ arrested = \dfrac{Black\ drug\ arrests}{Black\ population}$$

$$\%\ Latinx\ arrested = \dfrac{Latinx\ drug\ arrests}{Latinx\ population}$$

$$\%\ White\ arrested = \dfrac{White\ drug\ arrests}{White\ population}$$

We calculate the logit for each racial/ethnic group as:

$$logit(\%\ group\ arrested) = \log\left(\dfrac{\%\ group\ arrested}{1 - \%\ group\ arrested}\right)$$

Since the logit of the two percentages is now in log-space, we take the difference of the two quantities rather than the quotient (since $\log_b(x/y) = \log_b(x) - \log_b(y)$), to calculate the normalized disparity:

$$log(Black\text{-}White\ drug\ arrest\ disparity) = logit(\%\ Black\ arrested) - logit(\%\ White\ arrested)$$

Similarly for Latinx drug arrests:

$$log(Latinx\text{-}White\ drug\ arrest\ disparity) = logit(\%\ Latinx\ arrested) - logit(\%\ White\ arrested)$$

The log-disparity scores are now normalized so departments can effectively be compared with one another for each group. We then transform each department's log-disparity to a standard score ($\frac{x - \mu}{\sigma}$), which tells us how the department's disparity score compares to other departments across the state for a given population. A standard score of 2 for Latinx drug arrests would mean the department's Latinx disparity is 2 standard deviations higher than the normalized average of departments statewide, whereas a standard score of -2 would mean it's 2 standard deviations lower than average.

In [10]:
black_drug_arrests = raw['Black Drug Possession Arrests'].fillna(0)
latinx_drug_arrests = raw['Hispanic Drug Possession Arrests'].fillna(0)
white_drug_arrests = raw['White Drug Possession Arrests'].fillna(0)

total_drug_arrests = black_drug_arrests + latinx_drug_arrests + white_drug_arrests

In [11]:
black_pct_drug_arrests = black_drug_arrests / total_drug_arrests
pct_black_pop_arrested_drugs = black_drug_arrests / black_pop
white_pct_drug_arrests = white_drug_arrests / total_drug_arrests
pct_white_pop_arrested_drugs = white_drug_arrests / white_pop
latinx_pct_drug_arrests = latinx_drug_arrests[latinx_drug_arrests != 0] / total_drug_arrests
pct_latinx_pop_arrested_drugs = latinx_drug_arrests[latinx_drug_arrests != 0] / latinx_pop


In [12]:
logit_white_drug = numpy.log(pct_white_pop_arrested_drugs/(1-pct_white_pop_arrested_drugs))

In [13]:
logit_latinx_drug = numpy.log(pct_latinx_pop_arrested_drugs/(1-pct_latinx_pop_arrested_drugs))
logit_latinx_drug_arrest_disparity = (logit_latinx_drug - logit_white_drug)
latinx_drug_arrest_z_scores = (logit_latinx_drug_arrest_disparity - logit_latinx_drug_arrest_disparity.mean()) / logit_latinx_drug_arrest_disparity.std()

In [14]:
logit_black_drug = numpy.log(pct_black_pop_arrested_drugs/(1-pct_black_pop_arrested_drugs))
logit_black_drug_arrest_disparity = (logit_black_drug - logit_white_drug)
black_drug_arrest_z_scores = (logit_black_drug_arrest_disparity - logit_black_drug_arrest_disparity.mean()) / logit_black_drug_arrest_disparity.std()


In [15]:
hist(black_drug_arrest_z_scores)

In [16]:
drug_disparities = pandas.DataFrame({
    'agency_name': raw['Agency Name'],
    'pct_of_black_pop_arrested_drugs': black_drug_arrests / black_pop,
    'pct_of_hispanic_pop_arrested_drugs': latinx_drug_arrests / latinx_pop,
    'pct_of_white_pop_arrested_drugs': white_drug_arrests / white_pop,
    'black_drug_arrest_disparity': (black_drug_arrests / black_pop) / (white_drug_arrests / white_pop),
    'black_drug_arrest_z_score': black_drug_arrest_z_scores,
    'hispanic_drug_arrest_disparity': (latinx_drug_arrests / latinx_pop) / (white_drug_arrests / white_pop),
    'hispanic_drug_arrest_z_score': latinx_drug_arrest_z_scores,
})

drug_disparities['max_drug_arrest_disparity_score'] = drug_disparities[['black_drug_arrest_z_score', 'hispanic_drug_arrest_z_score']].fillna(0.0).max(axis=1)
drug_disparities['max_drug_arrest_disparity_score_percentile'] = 1.0 - drug_disparities.max_drug_arrest_disparity_score.rank(pct=True)


In [17]:
drug_disparities.sort_values('max_drug_arrest_disparity_score_percentile')

,agency_name,pct_of_black_pop_arrested_drugs,pct_of_hispanic_pop_arrested_drugs,pct_of_white_pop_arrested_drugs,black_drug_arrest_disparity,black_drug_arrest_z_score,hispanic_drug_arrest_disparity,hispanic_drug_arrest_z_score,max_drug_arrest_disparity_score,max_drug_arrest_disparity_score_percentile
6,Beverly Hills Police Department,0.104859,0.009438,0.001898,55.260870,3.697569,4.973575,2.951851,3.697569,0.00
47,Newport Beach Police Department,0.301075,0.018474,0.008043,37.432970,3.528331,2.296865,1.690028,3.528331,0.01
54,Palo Alto Police Department,0.068577,0.014192,0.002834,24.197792,2.708439,5.007625,2.969386,2.969386,0.02
5,Berkeley Police Department,0.039609,0.005185,0.002784,14.225807,2.067133,1.862074,1.332576,2.067133,0.03
44,Mountain View Police Department,0.025735,0.008885,0.003619,7.111958,1.258386,2.455356,1.790788,1.790788,0.04
86,Sunnyvale Police Department,0.017040,0.007866,0.003368,5.060031,0.859939,2.335793,1.707668,1.707668,0.05
48,Oakland Police Department,0.006526,0.001270,0.000710,9.190755,1.532143,1.787912,1.262908,1.532143,0.06
80,Santa Monica Police Department,0.019991,0.003728,0.002293,8.717792,1.485601,1.625791,1.108501,1.485601,0.07
32,Glendale Police Department,0.030485,0.013063,0.006540,4.661077,0.778269,1.997252,1.454328,1.454328,0.08
38,Livermore Police Department,0.054237,0.006085,0.006734,8.054450,1.430769,0.903591,0.142030,1.430769,0.09


# Police Use of Deadly Force

We begin this work from the premise that it is possible to live in a world where police don't kill people. Period. Even if the use of deadly force by police is a relatively "rare" event, it's all too common in the US compared to other countries, and there are [proven, well-researched policies](https://joincampaignzero.org) which significantly reduce police violence and improve accountability. Since this is the most traumatic and visible form of police violence, especially for Black and Latinx communities that are routinely and disproportionately criminalized and over-policed, we wanted to examine the additional disparities in deadly force given that an arrest took place. In Movement work, it is often said that if police treated everyone the way they treat white people, we would be closer to the world we want to see.

We define a model of that idea, specificially around the question: what would deaths by police look like in a given city *if the victims had been white*?

Imagine that every interaction with police is a single round of Russian roulette and white people are given two levels of advantage: first, they simply don't have to play the game as often, and when they do, they're given a different revolver with fewer chambers loaded. Both lead to disproportionate use of deadly force against communities of color. The first advantage (not having to interact with the police as much in the first place) is accounted for in the arrest disparity scores, and here we want to examine what happens *given* that an arrest did occur, and how that changes when a person of color is arrested.

We model use deadly force for each racial/ethnic group as a Binomial distribution:

$$Deadly\ force \sim \operatorname{Binomial} \left({n, p}\right)$$

The chances are very small that a white person will be killed by police during an arrest. For many police departments, there are no white people killed. For these departments, we don't think the chance is really 0. So, we use the statewide chances of a white person being killed during an arrest as a prior probability.


$$p = \frac{white\ residents\ killed\ by\ department + white\ residents\ killed\ by\ police\ statewide \cdot C}{white\ residents\ arrested\ by\ department + white\ residents\ arrested\ statewide \cdot C}$$

running $n$ Bernoulli trials for each Black or Latinx resident who was arrested.

$$n = \#\ of\ arrests\ in\ the\ population\ being\ considered\ (Black\ or\ Latinx)$$

We can then reason about the probability of observing the actual outcome (the number of Black and Latinx victims of deadly force) with

The mean of a Binomial distribution is:

$$\mu = np$$

the variance is:

$$\sigma^2 = np(1-p)$$

and the standard deviation is:

$$\sigma = \sqrt{np(1-p)}$$

Using the mean and standard deviation, we can convert the actual number of deaths by police $x$ for Black and Latinx residents to a standard score (Z-score) using the formula $\frac{x - \mu}{\sigma}$, which tells us how much the observed number of deaths for Black and Latinx residents deviates from what would happen in $n$ arrests of white residents.

**Note**: we use arrests instead of the population for this part of the analysis, since racial bias in arrests is already accounted for in other parts of the score (and there are cases like Oakland where, because they have adopted better deadly force policies, in 2016 they didn't kill people at nearly the same rates as other cities even given the fact that Oakland has a lot of work to do on reducing arrest disparities). This section of the score focuses on the disproportionate use of deadly force even when controlling for disparities in arrests. If we were instead looking at deadly force on its own, we could use population (the columns below tha end in `_by_pop`) instead of arrests to capture racial bias throughout the entire system.

In [18]:
black_arrests = raw['Black Arrests']
latinx_arrests = raw['Hispanic Arrests']
white_arrests = raw['White Arrests']

total_arrests =  black_arrests + latinx_arrests + white_arrests

black_deadly_force = raw['Black Victims of Deadly Force']
latinx_deadly_force = raw['Hispanic Victims of Deadly Force']
white_deadly_force = raw['White Victims of Deadly Force']
total_deadly_force = black_deadly_force + latinx_deadly_force + white_deadly_force

avg_deadly_force_per_white_population = float(white_deadly_force.sum()) / white_pop.sum()
avg_deadly_force_per_white_arrest = float(white_deadly_force.sum()) / white_arrests.sum()

# strength of prior beliefs
C_0 = 0.0001  # white_deadly_force_by_pop
C_1 = 0.001  # white_deadly_force_by_arrest

deadly_force = pandas.DataFrame({
    'agency_name': raw['Agency Name'],
    'black_actual': black_deadly_force,
    'hispanic_actual': latinx_deadly_force,
    'white_actual': white_deadly_force,
    'total_actual': total_deadly_force,
    'white_deadly_force_by_pop': numpy.exp(numpy.log(white_deadly_force + white_deadly_force.sum() * C_0) - numpy.log(white_pop + white_pop.sum() * C_0)),
    'white_deadly_force_by_arrest': numpy.exp(numpy.log(white_deadly_force + white_deadly_force.sum() * C_1) - numpy.log(white_arrests + white_arrests.sum() * C_1)),
})

deadly_force['black_mean_by_pop'] = black_pop * deadly_force.white_deadly_force_by_pop
deadly_force['black_variance_by_pop'] = black_pop * deadly_force.white_deadly_force_by_pop * (1 - deadly_force.white_deadly_force_by_pop)
deadly_force['black_stdev_by_pop'] = numpy.sqrt(deadly_force.black_variance_by_pop)
deadly_force['black_zscore_by_pop'] = (black_deadly_force - deadly_force.black_mean_by_pop) / deadly_force.black_stdev_by_pop

deadly_force['black_mean_by_arrest'] = black_arrests * deadly_force.white_deadly_force_by_arrest
deadly_force['black_variance_by_arrest'] = black_arrests * deadly_force.white_deadly_force_by_arrest * (1 - deadly_force.white_deadly_force_by_arrest)
deadly_force['black_stdev_by_arrest'] = numpy.sqrt(deadly_force.black_variance_by_arrest)
deadly_force['black_zscore_by_arrest'] = (black_deadly_force - deadly_force.black_mean_by_arrest) / deadly_force.black_stdev_by_arrest

deadly_force['hispanic_mean_by_pop'] = latinx_pop * deadly_force.white_deadly_force_by_pop
deadly_force['hispanic_variance_by_pop'] = latinx_pop * deadly_force.white_deadly_force_by_pop * (1 - deadly_force.white_deadly_force_by_pop)
deadly_force['hispanic_stdev_by_pop'] = numpy.sqrt(deadly_force.hispanic_variance_by_pop)
deadly_force['hispanic_zscore_by_pop'] = (latinx_deadly_force - deadly_force.hispanic_mean_by_pop) / deadly_force.hispanic_stdev_by_pop

deadly_force['hispanic_mean_by_arrest'] = latinx_arrests * deadly_force.white_deadly_force_by_arrest
deadly_force['hispanic_variance_by_arrest'] = latinx_arrests * deadly_force.white_deadly_force_by_arrest * (1 - deadly_force.white_deadly_force_by_arrest)
deadly_force['hispanic_stdev_by_arrest'] = numpy.sqrt(deadly_force.hispanic_variance_by_arrest)
deadly_force['hispanic_zscore_by_arrest'] = (latinx_deadly_force - deadly_force.hispanic_mean_by_arrest) / deadly_force.hispanic_stdev_by_arrest

deadly_force['deadly_force_score'] = deadly_force[['black_zscore_by_arrest', 'hispanic_zscore_by_arrest']].max(axis=1)
deadly_force['deadly_force_score_percentile'] = 1.0 - deadly_force.deadly_force_score.rank(pct=True)
deadly_force.sort_values('deadly_force_score')



,agency_name,black_actual,hispanic_actual,white_actual,total_actual,white_deadly_force_by_pop,white_deadly_force_by_arrest,black_mean_by_pop,black_variance_by_pop,black_stdev_by_pop,black_zscore_by_pop,black_mean_by_arrest,black_variance_by_arrest,black_stdev_by_arrest,black_zscore_by_arrest,hispanic_mean_by_pop,hispanic_variance_by_pop,hispanic_stdev_by_pop,hispanic_zscore_by_pop,hispanic_mean_by_arrest,hispanic_variance_by_arrest,hispanic_stdev_by_arrest,hispanic_zscore_by_arrest,deadly_force_score,deadly_force_score_percentile
48,Oakland Police Department,1,0,2,3,1.700245e-05,0.000625,1.709138,1.709109,1.307329,-0.542433,11.979703,11.972218,3.460089,-3.173243,1.941986,1.941953,1.393540,-1.393564,3.685230,3.682928,1.919096,-1.920295,-1.920295,0.99
18,Culver City Police Department,0,0,1,1,5.103876e-05,0.000838,0.159190,0.159182,0.398976,-0.398996,1.743569,1.742108,1.319889,-1.320996,0.469914,0.469890,0.685485,-0.685520,1.600313,1.598972,1.264505,-1.265565,-1.265565,0.98
73,San Leandro Police Department,0,0,1,1,5.156601e-05,0.000633,0.551344,0.551315,0.742506,-0.742544,1.467994,1.467065,1.211224,-1.211992,1.285798,1.285732,1.133901,-1.133960,1.092425,1.091733,1.044860,-1.045522,-1.045522,0.97
60,Redlands Police Department,0,0,4,4,1.053510e-04,0.001348,0.566789,0.566729,0.752814,-0.752893,1.029365,1.027977,1.013892,-1.015261,2.132305,2.132080,1.460164,-1.460318,3.403984,3.399396,1.843745,-1.846234,-1.015261,0.96
57,Pleasanton Police Department,0,0,2,2,4.752379e-05,0.001062,0.068054,0.068051,0.260866,-0.260878,0.611143,0.610494,0.781341,-0.782172,0.377149,0.377131,0.614110,-0.614139,1.020609,1.019525,1.009715,-1.010789,-0.782172,0.95
84,South San Francisco Police Department,0,1,2,3,1.518941e-04,0.001314,0.214474,0.214442,0.463079,-0.463149,0.604079,0.603285,0.776714,-0.777736,3.647736,3.647182,1.909760,-1.386424,2.340489,2.337413,1.528860,-0.876790,-0.777736,0.94
0,Alameda Police Department,0,0,1,1,3.015340e-05,0.000669,0.183272,0.183267,0.428097,-0.428110,0.661265,0.660823,0.812910,-0.813454,0.303524,0.303515,0.550922,-0.550939,0.505087,0.504749,0.710457,-0.710932,-0.710932,0.93
62,Redwood City Police Department,0,1,2,3,5.387498e-05,0.001018,0.102309,0.102303,0.319849,-0.319866,0.473799,0.473316,0.687980,-0.688681,1.774426,1.774331,1.332040,-0.581384,2.688449,2.685712,1.638814,-1.030287,-0.688681,0.92
68,Salinas Police Department,0,2,1,3,4.402906e-05,0.000446,0.077007,0.077003,0.277495,-0.277507,0.456434,0.456230,0.675448,-0.675749,5.412184,5.411946,2.326359,-1.466749,4.104462,4.102630,2.025495,-1.038986,-0.675749,0.91
41,Merced Police Department,0,1,1,2,3.972057e-05,0.000323,0.138029,0.138024,0.371515,-0.371530,0.617363,0.617163,0.785597,-0.785851,1.692573,1.692506,1.300963,-0.532354,1.845131,1.844535,1.358137,-0.622273,-0.622273,0.90


# Police Violence Score Calculation

In all our calculations, we grade departments on the highest (worst) of its scores between the Black and Latinx community in a city to ensure that racially biased policing against one marginalized group is not being erased or offset due to more equitable treatment of another group.

$$Max\ deadly\ force\ score = \max{(Black\ deadly\ force\ score,\ Latinx\ deadly\ force\ score)}$$

$$Max\ deadly\ force\ score = \max{(Black\ deadly\ force\ score,\ Latinx\ deadly\ force\ score)}$$

In [19]:
output = pandas.DataFrame()

output['police_department'] = raw['Agency Name']
output['black_drug_arrest_disparity'] = drug_disparities.black_drug_arrest_disparity
output['black_drug_arrest_score'] = drug_disparities.black_drug_arrest_z_score
output['hispanic_drug_arrest_disparity'] = drug_disparities.hispanic_drug_arrest_disparity
output['hispanic_drug_arrest_score'] = drug_disparities.hispanic_drug_arrest_z_score
output['drug_arrest_disparity_score'] = output[['black_drug_arrest_score', 'hispanic_drug_arrest_score']].max(axis=1)

output['black_deadly_force_score'] = deadly_force.black_zscore_by_arrest
output['hispanic_deadly_force_score'] = deadly_force.hispanic_zscore_by_arrest
output['deadly_force_score'] = output[['black_deadly_force_score', 'hispanic_deadly_force_score']].max(axis=1)

In [20]:
output.sort_values('drug_arrest_disparity_score')

,police_department,black_drug_arrest_disparity,black_drug_arrest_score,hispanic_drug_arrest_disparity,hispanic_drug_arrest_score,drug_arrest_disparity_score,black_deadly_force_score,hispanic_deadly_force_score,deadly_force_score
56,Pittsburg Police Department,0.835360,-1.213963,0.398239,-1.210118,-1.210118,1.606212,7.639461,7.639461
46,National City Police Department,1.097460,-0.898743,0.426813,-1.101865,-0.898743,2.181481,-0.799331,2.181481
23,Elk Grove Police Department,1.144792,-0.852002,0.092652,-3.594238,-0.852002,0.886272,2.896172,2.896172
4,Bakersfield Police Department,1.166227,-0.827043,0.392177,-1.246920,-0.827043,-0.097551,3.203757,3.203757
3,Antioch Police Department,1.293171,-0.711438,0.305692,-1.639933,-0.711438,0.793421,-1.298845,0.793421
98,Westminster Police Department,1.241135,-0.756042,0.545247,-0.696078,-0.696078,-0.138253,-0.389709,-0.138253
12,Chula Vista Police Department,1.261367,-0.739715,0.546314,-0.686727,-0.686727,-0.342009,0.572208,0.572208
50,Ontario Police Department,0.778789,-1.296446,0.551327,-0.679485,-0.679485,-0.451265,-0.836710,-0.451265
73,San Leandro Police Department,1.078481,-0.920058,0.575877,-0.598703,-0.598703,-1.211992,-1.045522,-1.045522
26,Fontana Police Department,0.628442,-1.541846,0.578675,-0.595462,-0.595462,0.466646,-0.148418,0.466646


In [21]:
output.sort_values('deadly_force_score')

,police_department,black_drug_arrest_disparity,black_drug_arrest_score,hispanic_drug_arrest_disparity,hispanic_drug_arrest_score,drug_arrest_disparity_score,black_deadly_force_score,hispanic_deadly_force_score,deadly_force_score
48,Oakland Police Department,9.190755,1.532143,1.787912,1.262908,1.532143,-3.173243,-1.920295,-1.920295
18,Culver City Police Department,2.808231,0.182230,1.836284,1.312536,1.312536,-1.320996,-1.265565,-1.265565
73,San Leandro Police Department,1.078481,-0.920058,0.575877,-0.598703,-0.598703,-1.211992,-1.045522,-1.045522
60,Redlands Police Department,1.170034,-0.825336,1.330928,0.783969,0.783969,-1.015261,-1.846234,-1.015261
57,Pleasanton Police Department,4.840223,0.808806,1.164516,0.559972,0.808806,-0.782172,-1.010789,-0.782172
84,South San Francisco Police Department,1.583497,-0.472616,0.405454,-1.184449,-0.472616,-0.777736,-0.876790,-0.777736
0,Alameda Police Department,1.897261,-0.274141,1.684697,1.166154,1.166154,-0.813454,-0.710932,-0.710932
62,Redwood City Police Department,3.346317,0.387053,0.859959,0.060685,0.387053,-0.688681,-1.030287,-0.688681
68,Salinas Police Department,4.650699,0.766027,0.599330,-0.532884,0.766027,-0.675749,-1.038986,-0.675749
41,Merced Police Department,2.048396,-0.176036,0.701124,-0.277774,-0.176036,-0.785851,-0.622273,-0.622273


In [22]:
output.to_excel('data/processed/police_violence_2016-2018.xlsx', index=False)